In [11]:
from Bio import *
import Bio.SeqIO
import os

In [28]:
# File to be analyzed
FASTA_FILE = ""
# Resulting file for aligned sequences
RESULT_FASTA_FILE = ""
# Resulting output of analysis
RESULT_CSV_FILE = ""

In [ ]:
# Aligning Sequences (or do it online)
os.system("mafft --6merpair --thread 6 --maxambiguous 0.05 --addfragments " + FASTA_FILE + " reference.fasta > " + RESULT_FASTA_FILE)

In [25]:
# Translating sequences using expasy

def translate(seq):
    translated = os.popen("curl -s -d \"dna_sequence=" + seq + "&output_format=fasta\" https://web.expasy.org/cgi-bin/translate/dna2aa.cgi").read()
    translated = translated.split(">")[4]
    return translated.split("\n", 1)[1].replace("\n", "")

In [26]:
# Selected refernece sequence (ex. DNA Polymerase of MPXV)

REF_SEQUENCE_2001 = """MDVRCINWFESHGENRFLYLKSRCRNGETVFIRFPHYFYYVVTDEIYQSLSPPPFNARPMGKMRTIDIDETISYNLDIKDRKCSVADMWLIEEPKKRSIQNATMDEFLNISWFYISNGISPDGCYSLDEQYLTKINNGCYHCDDPRNCFAKEIPRFDIPRSYLFLDIECHFDKKFPSVFINPISHTSYCYIDLSGKRLLFTLINEEMLTEQEIQEAVDRGCLRIQSLMEMDYERELVLCSEIVLLRIAKQLLELTFDYVVTFNGHNFDLRYITNRLELLTGEKIIFRSPDKKEAVHLCIYERNQSSHKGVCGMANTTFHVNNNNGTIFFDLYSFIQKSEKLDSYKLDSISKNAFSCMGKVLNRGVREMTFIGDDTTDAKGKADTFAKVLTTGNYVTVDEDIICKVIRKDIWENGFKVVLSCPTLPNDTYKLSFGKDDIDLAQMYKDYNLNIALDMARYCIHDACLCQYLWEYYGVETKTDAGASTYVLPQSMVFEYRASTVIKGPLLKLLLETKTILVRSETKQKFPYEGGKVFAPKQKMFSNNVLIFDYNSLYPNVCIFGNLSPETLVGVVVSTNRLEEEINNQLLLQKYPPPRYITVHCEPRLPNLISEIAIFDRSIEGTIPRLLRTFLAERARYKKMLKQATSSTEKAIYDSMQYTYKIVANSVYGLMGFRNSALYSYASAKSCTSIGRRMILYLESVLNGAELSNGMLRFANTLSNPFYMDDRDINPIVKTSLPIDYRFRFRSVYGDTDSVFTEIDSQDVDKSIEIAKELERLINSRVLFDNFKIEFEAVYKNLIMQSKKKYTTMKYSASSNSKSVPERINKGTSETRRDVSKFHKNMIKTYKTRLSEMLSEGRMNSNQVCIDILRSLETDLRSEFDSRSSPLELFMLSRMHHSNYKSADNPNMYLVTEYNKNNPETIELGERYYFAYICPANVPWTKKLVNIKTYETIIDRSFKLGSNQRIFYEVYFKRLTSEIVNLLDNKVLCISFFQRMFGSRPTFYEA"""

In [29]:
# Opening and initializing result file

csv_file = open(RESULT_CSV_FILE,  "a")
csv_file.write("ID,Date,Location,Mutations\n")
csv_file.close()

In [ ]:
# coordinates for selected protein
BEGIN = 53691
END = 56711
# Processing aligned sequences
total_seqs = 0
for seq_record in Bio.SeqIO.parse(RESULT_FASTA_FILE, "fasta"):
    date  = seq_record.description.split("|")[2]
    print("Date: " + date)
    dna_polymerase = seq_record.seq[BEGIN-1:END]
    print("ID: " + seq_record.id)
    print("Location: " + seq_record.description.split("|")[3])
    translated_sequence = translate(str(dna_polymerase))[:-1]
    mutations = []
    count = 0
    for i in REF_SEQUENCE_2001:
        try:
            if i != translated_sequence[count]:
                mutations.append(i + str(count + 1) + translated_sequence[count])
        except:
            pass
        count += 1
    print("Mutations: " + ";".join(mutations))
    csv_file = open(RESULT_CSV_FILE,  "a")
    csv_file.write(seq_record.id + "," + date + "," + seq_record.description.split("|")[3] + ",\"" + ";".join(mutations) + "\"\n")
    csv_file.close()
    total_seqs += 1
print("Number of sequences analyzed: " + str(total_seqs))